In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import getpass

In [ ]:
from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

ModuleNotFoundError: No module named 'sentinelhub'

In [ ]:
from utils import plot_image

In [ ]:
config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

In [ ]:
aoi_coords_wgs84 = [-2.410126,51.614233,-2.333908,51.650460]

In [ ]:
resolution = 10
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {aoi_size} pixels")

In [ ]:
catalog = SentinelHubCatalog(config=config)

In [ ]:
year = "2023"
month = "05"

In [ ]:
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
time_interval = year + month + "01", year + month + "31"
print (time_interval)

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=aoi_bbox,
    time=time_interval,
    fields={"include": ["id", "properties.datetime"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

results

In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

time_interval=("2023-05-01", "2023-05-31")

request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
            ),
            #time_interval=("2023-05-01", "2023-05-31"),
            # Feels odd below having time_interval twice, will revisit at some point...
            time_interval=time_interval,
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config,
)

In [ ]:
true_color_imgs = request_true_color.get_data()

In [ ]:
print(
    f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}."
)
print(
    f"Single element in the list is of type {type(true_color_imgs[-1])} and has shape {true_color_imgs[-1].shape}"
)

In [ ]:
image = true_color_imgs[0]
print(f"Image type: {image.dtype}")

# plot function
# factor 1/255 to scale between 0-1
# factor 3.5 to increase brightness
plot_image(image, factor=3.5 / 255, clip_range=(0, 1))

In [ ]:
evalscript_ndvi = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "dataMask"
      ]
    }],
    output: {
      bands: 4
    }
  }
}


function evaluatePixel(sample) {
    let val = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
    let imgVals = null;

    if (val<0.150) imgVals = [null];
    else if (val<0.175) imgVals = [1,1,0.88];
    else if (val<0.2) imgVals = [1,0.98,0.8];
    else if (val<0.25) imgVals = [0.93,0.91,0.71];
    else if (val<0.3) imgVals = [0.87,0.85,0.61];
    else if (val<0.35) imgVals = [0.8,0.78,0.51];
    else if (val<0.4) imgVals = [0.74,0.72,0.42];
    else if (val<0.45) imgVals = [0.69,0.76,0.38];
    else if (val<0.5) imgVals = [0.64,0.8,0.35];
    else if (val<0.55) imgVals = [0.57,0.75,0.32];
    else if (val<0.6) imgVals = [0.5,0.7,0.28];
    else if (val<0.65) imgVals = [0.44,0.64,0.25];
    else if (val<0.7) imgVals = [0.38,0.59,0.21];
    else if (val<0.75) imgVals = [0.31,0.54,0.18];
    else if (val<0.8) imgVals = [0.25,0.49,0.14];
    else if (val<0.85) imgVals = [0.19,0.43,0.11];
    else if (val<0.9) imgVals = [0.13,0.38,0.07];
    else if (val<0.95) imgVals = [0.06,0.33,0.04];
    else imgVals = [0,0.27,0];


    imgVals.push(sample.dataMask)

    return imgVals
}
"""

request_ndvi_img = SentinelHubRequest(
    evalscript=evalscript_ndvi,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
            ),
            time_interval=("2023-05-01", "2023-05-31"),
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config,
)

In [ ]:
ndvi_img = request_ndvi_img.get_data()

In [ ]:
print(
    f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}."
)
print(
    f"Single element in the list is of type {type(true_color_imgs[-1])} and has shape {true_color_imgs[-1].shape}"
)

In [ ]:
image = ndvi_img[0]
print(f"Image type: {image.dtype}")

# plot function
plot_image(image, factor=1 / 255)